# Part 1 V2
This script is meant to be an update to the original Part 1, simplifying the functions. Some changes to original part 1 code will be needed. 

In [14]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Download All of the Data

In [16]:
# Information
DATASOURCE_IDS = ['35', '38', '41', '42', '45']
j_drive_output_folder_path = 'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/'
geography_levels = ['mgra_id', 'mgra', 'cpa', 'LUZ', 'jurisdiction', 'region']

In [17]:
# download all of the data 
all_data = {}
for ds_id in DATASOURCE_IDS:
    temp_dict = {}
    for file in os.listdir(j_drive_output_folder_path):
        if (ds_id in file):
            temp_dict[file] = pd.read_csv(j_drive_output_folder_path + file)
    all_data[ds_id] = temp_dict

In [19]:
all_data['35'].keys()

dict_keys(['luz_DS35_ind_QA.csv', 'mgra_DS35_ind_QA.csv', 'region_DS35_ind_QA.csv'])

# Both Files

In [28]:
def create_both_file(DS1, DS2, level, send_to_j_drive):
    df1 = all_data[DS1][f"{level}_DS{DS1}_ind_QA.csv"]
    df2 = all_data[DS2][f"{level}_DS{DS2}_ind_QA.csv"]

    # First check to see if shapes are identical
    if df1.shape != df2.shape:
        print(f"Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}")
    elif len(set(df1.columns).difference(set(df2.columns))) != 0:
        print(f"There is a difference in columns. The columns that exist in DF1 but not in DF2 are {[x for x in df1.columns if x not in df2.columns]}. The columns that exist in DF2 but not in DF1 are {[x for x in df2.columns if x not in df1.columns]}")
    elif len(set(df1['yr_id']).difference(set(df2['yr_id']))) != 0:
        print(f"There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are {[x for x in list(set(df1['yr_id'])) if x not in list(set(df2['yr_id']))]}. The columns that exist in DF2 but not in DF1 are {[x for x in list(set(df2['yr_id'])) if x not in list(set(df1['yr_id']))]}")
    

    if level == 'region':
        output = df1.merge(df2, on=['year'], how='left', suffixes=[f'_DS{DS1}', f'_DS{DS2}'])
        output = output.groupby(['year']).sum()
    else:
        output = df1.merge(df2, on=[f'{level}','year'], how='left', suffixes=[f'_DS{DS1}', f'_DS{DS2}'])
        output = output.groupby([f'{level}', 'year']).sum()

    if send_to_j_drive == 'Y':
        output.to_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/both_files/' + f"{level}_both_DS{DS1}_DS{DS2}_QA.csv", index=True) 

    return output

In [31]:
both_file = create_both_file(DS1='35', DS2='41', level='mgra', send_to_j_drive='N')

Shapes are not identical. DF1 is shape (299026, 145) and DF2 is shape (299026, 148)


In [32]:
both_file

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra  year                                                                   
1     2016      3331       19          19           0           0       18   
      2018      3331       19          19           0           0       18   
      2020      3331       19          19           0           0       18   
      2023      3331       20          20           0           0       18   
      2025      3331       20          20           0           0       18   
...              ...      ...         ...         ...         ...      ...   
23002 2032      1254      120          20         100           0      100   
      2035      1254      120          20         100           0      100   
      2040      1254      120          20         100           0      106   
      2045      1254      120          20         100           0      107   
      2050      1254      120          20         100           0      109   

            hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra  year                                                   ...   
1     2016          18           0           0            0  ...   
      2018          18           0           0            0  ...   
      2020          18           0           0            0  ...   
      2023          18           0           0            0  ...   
      2025          18           0           0            0  ...   
...                ...         ...         ...          ...  ...   
23002 2032          14          86           0            0  ...   
      2035          14          86           0            0  ...   
      2040          16          90           0            0  ...   
      2045          17          90           0            0  ...   
      2050          17          92           0            0  ...   

            American Indian_DS41  Asian_DS41  Black_DS41  Hispanic_DS41  \
mgra  year                                                                
1     2016                   0.0         1.0         0.0            1.0   
      2018                   0.0         1.0         0.0           11.0   
      2020                   0.0         1.0         0.0            4.0   
      2023                   0.0         0.0         0.0            0.0   
      2025                   0.0         1.0         0.0            4.0   
...                          ...         ...         ...            ...   
23002 2032                   0.0         0.0         0.0            0.0   
      2035                   2.0        44.0         3.0           52.0   
      2040                   1.0        69.0         2.0           60.0   
      2045                   0.0        77.0         5.0           69.0   
      2050                   2.0        87.0         9.0           61.0   

            Other_DS41  Pacific Islander_DS41  Two or More_DS41  White_DS41  \
mgra  year                                                                    
1     2016         1.0                    0.0               2.0        36.0   
      2018         1.0                    0.0               2.0        35.0   
      2020         1.0                    0.0               6.0        31.0   
      2023         0.0                    0.0               0.0         0.0   
      2025         1.0                    0.0               2.0        34.0   
...                ...                    ...               ...         ...   
23002 2032         0.0                    0.0               0.0         0.0   
      2035         1.0                    1.0               3.0       133.0   
      2040         0.0                    1.0              14.0       109.0   
      2045         1.0                    0.0               9.0       102.0   
      2050         1.0                    1.0               6.0       101.0   

            Female_DS41  Male_DS41  
mgra  year                          
1     2016         23.0       18.0

# Diff Files

In [64]:
def create_diff_file(DS1, DS2, level, send_to_j_drive):
    df1 = all_data[DS1][f"forecast_{DS1}_{level}_ind.csv"]
    df2 = all_data[DS2][f"forecast_{DS2}_{level}_ind.csv"]

    # First check to see if shapes are identical
    if df1.shape != df2.shape:
        print("Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}")

    # Check if different columns are present amongst datasets, if so, manipulate so that only the shared columns are present
    if len(set(df1.columns).difference(set(df2.columns))) != 0:
        print(f"There is a difference in columns. The columns that exist in DF1 but not in DF2 are {[x for x in df1.columns if x not in df2.columns]}. The columns that exist in DF2 but not in DF1 are {[x for x in df2.columns if x not in df1.columns]}")
        # Ensure each data frame has the same columns
        df1 = df1[[x for x in df1.columns if x in df2.columns]]
        df2 = df2[[x for x in df1.columns if x in df2.columns]]

    # Check if different years are present amongst datasets, if so, manipulate so that only the shared years are compared
    if len(set(df1['yr_id']).difference(set(df2['yr_id']))) != 0:
        print(f"There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are {[x for x in list(set(df1['yr_id'])) if x not in list(set(df2['yr_id']))]}. The columns that exist in DF2 but not in DF1 are {[x for x in list(set(df2['yr_id'])) if x not in list(set(df1['yr_id']))]}")
        shared_years = [x for x in list(set(df1['yr_id'])) if x in list(set(df2['yr_id']))]
        df1 = df1[df1['yr_id'].isin(shared_years)]
        df2 = df2[df2['yr_id'].isin(shared_years)]
    
    df1 = df1.groupby([f'{level}', 'yr_id']).sum()
    df2 = df2.groupby([f'{level}', 'yr_id']).sum()

    diff_df = df2 - df1


    if send_to_j_drive == 'Y':
        diff_df.to_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs/' + f"{level}_diff_DS{DS2}_minus_DS{DS1}_QA.csv", index=True) 

    return diff_df

In [67]:
result = create_diff_file(DS1='35', DS2='41', level='cpa', send_to_j_drive='N')

In [68]:
result

10 to 14  15 to 17  18 and 19  20 to 24  25 to 29  \
cpa             yr_id                                                      
*Not in a CPA*  2016          0         0          0         0         0   
                2018        198      -260        167        53       230   
                2020       -169      -139       -154        43       252   
                2025       -524        36        -77       479       797   
                2030       -114       116       -192      -339      -263   
...                         ...       ...        ...       ...       ...   
Via De La Valle 2030         10         0         -6        -5         2   
                2035        -23         9         -5        -7       -14   
                2040        -14         6         -9        -9         0   
                2045        -11         1          7         2        -9   
                2050        -13        -4          2         0         7   

                       30 to 34  35 to 39  40 to 44  45 to 49  5 to 9  ...  \
cpa             yr_id                                                  ...   
*Not in a CPA*  2016          0         0         0         0       0  ...   
                2018        132       105      -105        71     334  ...   
                2020       -170      -210       118       161     215  ...   
                2025        220      -285        82      -226    -186  ...   
                2030        167       630       272       218    -365  ...   
...                         ...       ...       ...       ...     ...  ...   
Via De La Valle 2030         15         9        -7        -9     -26  ...   
                2035        -11        12        15         7      -4  ...   
                2040          8       -12         7        16       2  ...   
                2045         11         7         1         2       1  ...   
                2050         -4         7        -5         6       0  ...   

                       Retail Trade  Self-Employed  \
cpa             yr_id                                
*Not in a CPA*  2016           -831              0   
                2018           -923              3   
                2020           -852              6   
                2025           -870             28   
                2030           -857             -5   
...                             ...            ...   
Via De La Valle 2030              0              0   
                2035              0              0   
                2040              0              0   
                2045              0             -1   
                2050              0             -1   

                       Transporation, Warehousing, and Utilities  \
cpa             yr_id                                              
*Not in a CPA*  2016                                        -160   
                2018                                        -155   
                2020                                        -156   
                2025                                        -181   
                2030                                        -152   
...                                                          ...   
Via De La Valle 2030                                           0   
                2035                                           0   
                2040                                           0   
                2045                                           0   
                2050                                           0   

                       Wholesale Trade  Group Quarters - College  \
cpa             yr_id                                              
*Not in a CPA*  2016                32                         0   
                2018                -1                         0   
                2020                -1                         0   
                2025                 4                         0   
                2030   